In [ ]:
import strawberryfields as sf
import numpy as np
import matplotlib.pyplot as plt
from typing import List, Tuple, Dict
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from scipy.stats import skew, kurtosis
from itertools import product
import multiprocessing as mp
import json
import os
import seaborn as sns
from tqdm import tqdm   # <-- Added import for progress bar

# ==============================================================================
# CONFIGURATION: ALL PARAMETERS USED IN THE PIPELINE
# ==============================================================================
PIPELINE_CONFIG = {
    "sf_hbar": 1,
    "quad_range": (-6, 6),
    "quad_points": 200,  # for state simulation Wigner, not directly for measurements
    "num_bins": 100,     # not used anymore for feature extraction
    "num_shots": 100,    # homodyne measurements per grid point (not used for these marginal features)
    "grid": {
        "num_db": 20,       # distinct squeezing values (in dB)
        "num_loss": 20,     # distinct loss values
        "num_gamma": 20     # distinct dephasing values
    },
    "db_range": (10.0, 13.0),   # dB values range
    "loss_range": (0.8, 1.0),     # loss values range
    "gamma_range": (0.8, 1.0),    # dephasing values range
    "model": {
        "input_shape": (10,),        
        "loss_bins": 20,
        "dephasing_bins": 20,
        "learning_rate": 1e-3,
        "epochs": 1000,
        "batch_size": 32,
        "validation_split": 0.2
    },
    "save_paths": {
        "dataset": "dataset.npz",
        "model": "trained_model.h5",
        "config": "pipeline_config.json"
    }
}

# Save the pipeline configuration to disk.
with open(PIPELINE_CONFIG["save_paths"]["config"], "w") as f:
    json.dump(PIPELINE_CONFIG, f, indent=4)

# ==============================================================================
# SETUP: CONFIGURATION PARAMETERS AND GLOBAL VARIABLES
# ==============================================================================
print("Step 1: Setting configuration parameters.")
CONFIG = {
    "hbar": PIPELINE_CONFIG["sf_hbar"],
    "quad_range": PIPELINE_CONFIG["quad_range"],
    "quad_points": PIPELINE_CONFIG["quad_points"],
    "cmap": plt.cm.RdBu,
    "colors": {
        "q_marginal": "navy",
        "p_marginal": "maroon",
        "samples_x": "skyblue",
        "samples_p": "lightcoral",
        "ideal": "red",
        "noisy": "green"
    }
}
sf.hbar = CONFIG["hbar"]
CONFIG["scale"] = np.sqrt(CONFIG["hbar"] * np.pi)
print("Configuration set. Scale =", CONFIG["scale"])

# ==============================================================================
# UTILITY: dB to epsilon conversion and feature extraction
# ==============================================================================
def db_to_epsilon(db_val: float) -> float:
    """
    Convert a given GKP squeezing level in dB to epsilon via:
         tanh(epsilon) = 10^(-db_val/10)
    """
    t = 10.0 ** (-db_val / 10.0)
    eps = 0.5 * np.log((1.0 + t) / (1.0 - t))
    return eps

def extract_statistical_features(samples_x: np.ndarray, samples_p: np.ndarray) -> np.ndarray:
    """Extract mean, variance, skewness, and kurtosis for X and P quadratures."""
    x_features = [
        np.mean(samples_x), np.var(samples_x),
        skew(samples_x), kurtosis(samples_x)
    ]
    p_features = [
        np.mean(samples_p), np.var(samples_p),
        skew(samples_p), kurtosis(samples_p)
    ]
    return np.array(x_features + p_features)

def extract_marginal_features(prep_state: List[float], epsilon: float, noise_params: Dict) -> np.ndarray:
    """
    Extract features from the marginal distributions:
      - Compute ideal and noisy marginals (for both X and P)
      - Compute statistical moments (mean, var, skew, kurtosis) from the noisy marginals
      - Compute a difference metric (mean absolute difference) between ideal and noisy marginals
    Returns a feature vector of length 10.
    """
    # Compute marginals using provided functions.
    ideal_x, ideal_p = calculate_ideal_marginals(prep_state, epsilon)
    noisy_x, noisy_p = calculate_noisy_marginals(prep_state, epsilon, noise_params)
    
    # Compute moments for the noisy marginals.
    features = [
        np.mean(noisy_x), np.var(noisy_x), skew(noisy_x), kurtosis(noisy_x),
        np.mean(noisy_p), np.var(noisy_p), skew(noisy_p), kurtosis(noisy_p)
    ]
    # Compute difference metrics (mean absolute difference).
    diff_x = np.mean(np.abs(ideal_x - noisy_x))
    diff_p = np.mean(np.abs(ideal_p - noisy_p))
    features.extend([diff_x, diff_p])
    
    return np.array(features)

# ==============================================================================
# CORE SIMULATION FUNCTIONS
# ==============================================================================
def simulate_gkp(
    prep_state: List[float],
    epsilons: List[float],
    noise_params: Dict = None,
    num_samples: int = 1
) -> Tuple[List[np.ndarray], List[np.ndarray], List[np.ndarray]]:
    """
    Simulate GKP states with optional noise channels.
    Returns lists of Wigner functions and marginals for each epsilon.
    """
    noise_params = noise_params or {}
    quad = np.linspace(*CONFIG["quad_range"], CONFIG["quad_points"]) * CONFIG["scale"]
    
    wigners, marginals_q, marginals_p = [], [], []
    
    for epsilon in epsilons:
        print(f"Processing ε={epsilon}...")
        avg_wigner = np.zeros((len(quad), len(quad)))
        
        for _ in range(num_samples):
            prog = sf.Program(1)
            with prog.context as q:
                sf.ops.GKP(state=prep_state, epsilon=epsilon) | q
                if 'loss' in noise_params:
                    sf.ops.LossChannel(noise_params['loss']) | q
                if 'gamma' in noise_params:
                    theta = np.random.normal(0, np.sqrt(2 * noise_params['gamma']))
                    sf.ops.Rgate(theta) | q
            eng = sf.Engine("bosonic")
            state = eng.run(prog).state
            if num_samples > 1:
                avg_wigner += state.wigner(0, quad, quad)
            else:
                avg_wigner = state.wigner(0, quad, quad)
        
        wigners.append(avg_wigner / num_samples if num_samples > 1 else avg_wigner)
        marginals_q.append(state.marginal(0, quad, phi=0))
        marginals_p.append(state.marginal(0, quad, phi=np.pi/2))
        
    return wigners, marginals_q, marginals_p

def simulate_homodyne(prep_state: List[float], epsilon: float, noise_params: Dict, num_samples: int) -> Tuple[np.ndarray, np.ndarray]:
    """
    Simulate homodyne measurements (X and P quadratures) for a given epsilon and noise parameters.
    Returns two arrays of measurement outcomes.
    """
    samples_x, samples_p = [], []
    for _ in range(num_samples):
        prog_x = sf.Program(1)
        with prog_x.context as q:
            sf.ops.GKP(state=prep_state, epsilon=epsilon) | q
            if 'loss' in noise_params:
                sf.ops.LossChannel(noise_params['loss']) | q
            if 'gamma' in noise_params:
                theta = np.random.normal(0, np.sqrt(2 * noise_params['gamma']))
                sf.ops.Rgate(theta) | q
            sf.ops.MeasureX | q
        eng_x = sf.Engine("bosonic")
        sample_x = eng_x.run(prog_x).samples[0, 0] / CONFIG["scale"]
        samples_x.append(sample_x)
        
        prog_p = sf.Program(1)
        with prog_p.context as q:
            sf.ops.GKP(state=prep_state, epsilon=epsilon) | q
            if 'loss' in noise_params:
                sf.ops.LossChannel(noise_params['loss']) | q
            if 'gamma' in noise_params:
                theta = np.random.normal(0, np.sqrt(2 * noise_params['gamma']))
                sf.ops.Rgate(theta) | q
            sf.ops.MeasureP | q
        eng_p = sf.Engine("bosonic")
        sample_p = eng_p.run(prog_p).samples[0, 0] / CONFIG["scale"]
        samples_p.append(sample_p)
    
    return np.array(samples_x), np.array(samples_p)

def calculate_ideal_marginals(prep_state: List[float], epsilon: float) -> Tuple[np.ndarray, np.ndarray]:
    """Calculate ideal marginal distributions."""
    prog = sf.Program(1)
    with prog.context as q:
        sf.ops.GKP(state=prep_state, epsilon=epsilon) | q
    state = sf.Engine("bosonic").run(prog).state
    quad_axis = np.linspace(-6, 6, 1000) * CONFIG["scale"]
    return (
        state.marginal(0, quad_axis, phi=0) * CONFIG["scale"],
        state.marginal(0, quad_axis, phi=np.pi/2) * CONFIG["scale"]
    )

def calculate_noisy_marginals(prep_state: List[float], epsilon: float, noise_params: Dict) -> Tuple[np.ndarray, np.ndarray]:
    """Calculate averaged noisy marginals."""
    quad_axis = np.linspace(-6, 6, 1000) * CONFIG["scale"]
    num_avg = 100
    marg_x, marg_p = np.zeros_like(quad_axis), np.zeros_like(quad_axis)
    
    for _ in range(num_avg):
        prog = sf.Program(1)
        with prog.context as q:
            sf.ops.GKP(state=prep_state, epsilon=epsilon) | q
            if 'loss' in noise_params:
                sf.ops.LossChannel(noise_params['loss']) | q
            if 'gamma' in noise_params:
                theta = np.random.normal(0, np.sqrt(2 * noise_params['gamma']))
                sf.ops.Rgate(theta) | q
        state = sf.Engine("bosonic").run(prog).state
        marg_x += state.marginal(0, quad_axis, phi=0)
        marg_p += state.marginal(0, quad_axis, phi=np.pi/2)
        
    return (marg_x/num_avg * CONFIG["scale"], marg_p/num_avg * CONFIG["scale"])

def plot_quadratures(
    samples_x: np.ndarray,
    samples_p: np.ndarray,
    ideal_x: np.ndarray,
    ideal_p: np.ndarray,
    noisy_x: np.ndarray,
    noisy_p: np.ndarray,
    title: str = "",
    figsize: Tuple[float, float] = (12, 5)
):
    """Plot quadrature measurement results"""
    quad_axis = np.linspace(-6, 6, 1000)
    fig, axs = plt.subplots(1, 2, figsize=figsize)
    
    # X quadrature
    axs[0].hist(samples_x, bins=100, density=True, 
               color=CONFIG["colors"]["samples_x"], label="Samples")
    # axs[0].plot(quad_axis, ideal_x, '--', color=CONFIG["colors"]["ideal"], label="Ideal")
    axs[0].plot(quad_axis, noisy_x, '-', color=CONFIG["colors"]["noisy"], label="Noisy")
    axs[0].set_xlabel(r"$q$ ($\sqrt{\pi\hbar}$)", fontsize=12)
    axs[0].set_ylabel("Probability Density", fontsize=12)

    # Add bin overlays for X quadrature
    # Adjust j range to cover the plotted quad_axis (here, -3 to 3 covers roughly -6 to 6)
    for j in range(-3, 4):
        axs[0].axvspan(2*j - 0.5, 2*j + 0.5, alpha=0.2, facecolor='b')
        axs[0].axvspan(2*j + 0.5, 2*j + 1.5, alpha=0.2, facecolor='r')
    
    # P quadrature
    axs[1].hist(samples_p, bins=100, density=True, 
               color=CONFIG["colors"]["samples_p"], label="Samples")
    # axs[1].plot(quad_axis, ideal_p, '--', color=CONFIG["colors"]["ideal"], label="Ideal")
    axs[1].plot(quad_axis, noisy_p, '-', color=CONFIG["colors"]["noisy"], label="Noisy")
    axs[1].set_xlabel(r"$p$ ($\sqrt{\pi\hbar}$)", fontsize=12)

    # Add bin overlays for P quadrature
    for j in range(-3, 4):
        axs[1].axvspan(2*j - 0.5, 2*j + 0.5, alpha=0.2, facecolor='b')
        axs[1].axvspan(2*j + 0.5, 2*j + 1.5, alpha=0.2, facecolor='r')
    
    for ax in axs:
        ax.legend()
        ax.tick_params(axis='both', which='major', labelsize=10)
    
    if title:
        fig.suptitle(title, y=1.02)
    plt.tight_layout()
    plt.show()

# ==============================================================================
# DEFINE THE FEED-FORWARD NEURAL NETWORK (FNN) MODEL
# ==============================================================================
def build_fnn_model_ver1(input_shape: Tuple[int], loss_bins: int, dephasing_bins: int) -> Model:
    """Build a feed-forward neural network for loss and dephasing prediction."""
    inputs = Input(shape=input_shape, name='statistical_features')
    x = Dense(128, activation='relu')(inputs)
    x = Dropout(0.3)(x)
    x = Dense(64, activation='relu')(x)
    x = Dropout(0.3)(x)
    
    loss_output = Dense(loss_bins, activation='softmax', name='loss_output')(x)
    dephasing_output = Dense(dephasing_bins, activation='softmax', name='dephasing_output')(x)
    
    model = Model(inputs=inputs, outputs=[loss_output, dephasing_output])
    return model

def build_fnn_model(input_shape: Tuple[int], loss_bins: int, dephasing_bins: int) -> Model:
    """
    Build a deeper feed-forward neural network for loss and dephasing prediction.
    """
    inputs = Input(shape=input_shape, name='statistical_features')
    
    x = Dense(256, activation='relu')(inputs)
    x = Dropout(0.3)(x)
    
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.3)(x)
    
    x = Dense(64, activation='relu')(x)
    x = Dropout(0.3)(x)
    
    loss_output = Dense(loss_bins, activation='softmax', name='loss_output')(x)
    dephasing_output = Dense(dephasing_bins, activation='softmax', name='dephasing_output')(x)
    
    model = Model(inputs=inputs, outputs=[loss_output, dephasing_output])
    return model

# ==============================================================================
# CREATE A GRID OF DISTINCT VALUES AND GENERATE THE DATASET
# ==============================================================================
print("Step 3: Preparing grid parameters for dataset generation.")

grid_config = PIPELINE_CONFIG["grid"]
num_db = grid_config["num_db"]
num_loss = grid_config["num_loss"]
num_gamma = grid_config["num_gamma"]

# Define the parameter ranges.
db_values = np.linspace(PIPELINE_CONFIG["db_range"][0], PIPELINE_CONFIG["db_range"][1], num_db)
loss_values = np.linspace(PIPELINE_CONFIG["loss_range"][0], PIPELINE_CONFIG["loss_range"][1], num_loss)
gamma_values = np.linspace(PIPELINE_CONFIG["gamma_range"][0], PIPELINE_CONFIG["gamma_range"][1], num_gamma)

# Define the two possible GKP preparation states.
prep_states = [[0, 0], [np.pi, 0]]

# Create a grid over all parameters: (prep_state, db, loss, gamma)
grid_params = list(product(prep_states, db_values, loss_values, gamma_values))

# For labeling, we use grid indices.
loss_bins = num_loss
dephasing_bins = num_gamma

def process_grid_point(params):
    prep_state, db_val, loss_val, gamma_val = params
    epsilon = db_to_epsilon(db_val)
    current_noise_params = {"loss": loss_val, "gamma": gamma_val}
    # Instead of simulating theirhomodyne samples and extracting  moments,
    # we extract features from the marginal distributions.
    features = extract_marginal_features(prep_state, epsilon, current_noise_params)
    # Determine grid indices for labels.
    loss_idx = np.where(np.isclose(loss_values, loss_val))[0][0]
    dephasing_idx = np.where(np.isclose(gamma_values, gamma_val))[0][0]
    return features, loss_idx, dephasing_idx


Step 1: Setting configuration parameters.
Configuration set. Scale = 1.7724538509055159
Step 3: Preparing grid parameters for dataset generation.


In [1]:
if __name__ == '__main__':
    dataset_path = PIPELINE_CONFIG["save_paths"]["dataset"]
    print("Generating examples")
    
    if not os.path.exists(dataset_path):
        print("No dataset found. Generating new dataset...")
        results = []
        for params in tqdm(grid_params, total=len(grid_params)):
            results.append(process_grid_point(params))
        
        X_data, y_loss, y_dephasing = zip(*results)
        X_data = np.array(X_data)  # shape: (num_examples, 10)
        y_loss = np.array(y_loss)
        y_dephasing = np.array(y_dephasing)
        num_examples = X_data.shape[0]
        print("Total examples generated:", num_examples)
        
        # Save the dataset.
        np.savez_compressed(PIPELINE_CONFIG["save_paths"]["dataset"],
                            X_data=X_data, y_loss=y_loss, y_dephasing=y_dephasing)
        print("Dataset saved to", PIPELINE_CONFIG["save_paths"]["dataset"])
    
    else:
        print(f"Dataset file found. Skipping dataset generation.")
        
        # Load dataset directly
        data = np.load(dataset_path)
        X_data = data["X_data"]
        print(f"Original dataset shape {X_data.shape}")
        X_data = X_data[:, :8]
        print(f"Droped marginal dataset shape {X_data.shape}")
        y_loss = data["y_loss"]
        y_dephasing = data["y_dephasing"]
        num_examples = X_data.shape[0]
        print("Dataset loaded. Total examples:", num_examples)
        
    # ==============================================================================
    # SPLIT THE DATASET INTO TRAINING AND TEST SETS (80/20 split)
    # ==============================================================================
    print("Step 4: Splitting dataset into training and test sets.")
    split = int(0.8 * num_examples)
    X_train, X_test = X_data[:split], X_data[split:]
    y_loss_train, y_loss_test = y_loss[:split], y_loss[split:]
    y_dephasing_train, y_dephasing_test = y_dephasing[:split], y_dephasing[split:]
    print("Training examples:", X_train.shape[0])
    print("Test examples:", X_test.shape[0])

    # ==============================================================================
    # BUILD AND COMPILE THE FNN MODEL
    # ==============================================================================
    print("Step 5: Building and compiling the FNN model.")
    model_config = PIPELINE_CONFIG["model"]
    input_shape = model_config["input_shape"]
    model = build_fnn_model(input_shape, model_config["loss_bins"], model_config["dephasing_bins"])
    model.compile(optimizer=Adam(learning_rate=model_config["learning_rate"]),
                  loss={'loss_output': 'sparse_categorical_crossentropy',
                        'dephasing_output': 'sparse_categorical_crossentropy'},
                  metrics={'loss_output': 'accuracy',
                           'dephasing_output': 'accuracy'})
    model.summary()

    # ==============================================================================
    # TRAIN THE MODEL
    # ==============================================================================
    print("Step 6: Training the model.")
    history = model.fit(X_train, 
                        {'loss_output': y_loss_train, 'dephasing_output': y_dephasing_train},
                        epochs=model_config["epochs"], 
                        batch_size=model_config["batch_size"], 
                        validation_split=model_config["validation_split"], 
                        verbose=1)

    # Save the trained model.
    model.save(PIPELINE_CONFIG["save_paths"]["model"])
    print("Trained model saved to", PIPELINE_CONFIG["save_paths"]["model"])

    # ==============================================================================
    # PLOT TRAINING ACCURACY AND LOSS CURVES
    # ==============================================================================
    print("Step 7: Plotting training accuracy and loss curves.")
    plt.figure(figsize=(12, 5))
    plt.plot(history.history['loss_output_accuracy'], label='Train Loss Accuracy')
    plt.plot(history.history['val_loss_output_accuracy'], label='Val Loss Accuracy')
    plt.plot(history.history['dephasing_output_accuracy'], label='Train Dephasing Accuracy')
    plt.plot(history.history['val_dephasing_output_accuracy'], label='Val Dephasing Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.title('Model Accuracy During Training')
    plt.legend()
    plt.show()

    plt.figure(figsize=(12, 5))
    plt.plot(history.history['loss'], label='Train Total Loss')
    plt.plot(history.history['val_loss'], label='Val Total Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Model Loss During Training')
    plt.legend()
    plt.show()

    # ==============================================================================
    # EVALUATE THE MODEL ON THE TEST SET
    # ==============================================================================
    print("Step 8: Evaluating the model on the test set.")
    def evaluate_model(model, X_test, y_loss_test, y_dephasing_test):
        pred_loss_probs, pred_dephasing_probs = model.predict(X_test)
        pred_loss = np.argmax(pred_loss_probs, axis=1)
        pred_dephasing = np.argmax(pred_dephasing_probs, axis=1)
        
        loss_accuracy = accuracy_score(y_loss_test, pred_loss)
        dephasing_accuracy = accuracy_score(y_dephasing_test, pred_dephasing)
        
        print("Loss Prediction Accuracy: {:.2f}%".format(loss_accuracy * 100))
        print("Dephasing Prediction Accuracy: {:.2f}%".format(dephasing_accuracy * 100))
        
        print("\nLoss Classification Report:")
        print(classification_report(y_loss_test, pred_loss))
        print("\nDephasing Classification Report:")
        print(classification_report(y_dephasing_test, pred_dephasing))
        
        # Plot confusion matrices
        fig, axes = plt.subplots(1, 2, figsize=(12, 5))
        cm_loss = confusion_matrix(y_loss_test, pred_loss)
        sns.heatmap(cm_loss, annot=True, fmt='d', ax=axes[0], cmap='Blues')
        axes[0].set_title("Loss Parameter Confusion Matrix")
        axes[0].set_xlabel("Predicted Bin")
        axes[0].set_ylabel("True Bin")
        
        cm_dephasing = confusion_matrix(y_dephasing_test, pred_dephasing)
        sns.heatmap(cm_dephasing, annot=True, fmt='d', ax=axes[1], cmap='Blues')
        axes[1].set_title("Dephasing Parameter Confusion Matrix")
        axes[1].set_xlabel("Predicted Bin")
        axes[1].set_ylabel("True Bin")
        plt.tight_layout()
        plt.show()
        
    evaluate_model(model, X_test, y_loss_test, y_dephasing_test)

    # ==============================================================================
    # BAYESIAN UPDATE FUNCTION FOR COMBINING MULTIPLE INDEPENDENT MEASUREMENTS
    # ==============================================================================
    print("Step 9: Defining Bayesian update function.")
    def bayesian_update(model, X_samples, y_loss_true=None, y_dephasing_true=None, 
                        loss_bin_centers=None, dephasing_bin_centers=None):
        """
        Given a list of independent FNN inputs (X_samples) corresponding to repeated measurements
        for the same channel setting, compute the combined posterior probability for each output.
        Optionally print estimated and true parameter values if bin centers and true labels are provided.
        """
        pred_loss_probs, pred_dephasing_probs = model.predict(X_samples[0][np.newaxis, ...])
        combined_loss_post = pred_loss_probs.flatten()
        combined_dephasing_post = pred_dephasing_probs.flatten()
        
        for X in X_samples[1:]:
            pred_loss, pred_dephasing = model.predict(X[np.newaxis, ...])
            combined_loss_post *= pred_loss.flatten()
            combined_dephasing_post *= pred_dephasing.flatten()
        
        combined_loss_post /= np.sum(combined_loss_post)
        combined_dephasing_post /= np.sum(combined_dephasing_post)
        
        est_loss_bin = np.argmax(combined_loss_post)
        est_dephasing_bin = np.argmax(combined_dephasing_post)
        
        print("Bayesian Update: MAP estimate for Loss Bin:", est_loss_bin)
        print("Bayesian Update: MAP estimate for Dephasing Bin:", est_dephasing_bin)
        
        if loss_bin_centers is not None and y_loss_true is not None:
            est_loss_value = loss_bin_centers[est_loss_bin]
            true_loss_value = loss_bin_centers[y_loss_true]
            print("Estimated Loss Value: {:.4f} (True: {:.4f})".format(est_loss_value, true_loss_value))
        if dephasing_bin_centers is not None and y_dephasing_true is not None:
            est_dephasing_value = dephasing_bin_centers[est_dephasing_bin]
            true_dephasing_value = dephasing_bin_centers[y_dephasing_true]
            print("Estimated Dephasing Value: {:.4f} (True: {:.4f})".format(est_dephasing_value, true_dephasing_value))
        
        return combined_loss_post, combined_dephasing_post

    # ==============================================================================
    # EXAMPLE USAGE OF THE BAYESIAN UPDATE AFTER TRAINING
    # ==============================================================================
    print("Step 10: Demonstrating Bayesian update on repeated measurements.")
    X_test_list = []
    num_bayes_samples = 5  # number of independent measurements
    
    # For demonstration, use the parameters of the first test example.
    db_val_example = db_values[0]
    loss_val_example = loss_values[y_loss_test[0]]
    gamma_val_example = gamma_values[y_dephasing_test[0]]
    epsilon_example = db_to_epsilon(db_val_example)
    current_noise_params_example = {"loss": loss_val_example, "gamma": gamma_val_example}
    
    demo_prep_state = [0, 0]
    
    for _ in range(num_bayes_samples):
        # We use the marginal-feature extraction here too.
        features = extract_marginal_features(demo_prep_state, epsilon_example, current_noise_params_example)
        X_test_list.append(features)
    combined_loss_post, combined_dephasing_post = bayesian_update(
        model, X_test_list, 
        y_loss_true=y_loss_test[0], y_dephasing_true=y_dephasing_test[0],
        loss_bin_centers=loss_values, dephasing_bin_centers=gamma_values
    )

NameError: name 'PIPELINE_CONFIG' is not defined